# MPC Model Training

The purpose of this notebook is to take data collected from the MPC data collection process in notebooks c and d and use it to train a gaussian process model of the dynamics of the tank system.  The models to be developed are in the form:

$$
\frac{dh_i}{dt} = f_i(h_1, h_2, u)
$$

where
- $i$ - the tank number
- $h_i$ - the tank height
- $u$ - the control vale position

For reference, when collecting data, the PWM value is set as soon as the data is sampled and the change between that and the next step is what tells how that PWM value effected the system.

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import joblib

# Import data
df_initial = pd.read_excel('data/MPC_Initial_Data.xlsx')
# df_operations = pd.read_excel('data/MPC_Operations_Data_2.xlsx')
df_pid = pd.read_excel('data/pid_data.xlsx')
df = pd.concat([df_initial, df_pid], ignore_index=True).reset_index()

The data collected is in terms of just the raw arduino outputs from the data collection process.  Processing of the data needs to happen to calculate the derivatives and make sure the dynamics is assigned to the correct row.

In [ ]:
# Calculate the time difference between consecutive rows in seconds
df['time_diff_seconds'] = df['Timestamp'].diff().dt.total_seconds()

# Calculate the finite difference (rate of change) for both tanks
df['dHeight_Tank1_dt'] = df['Height Tank 1'].diff() / df['time_diff_seconds']
df['dHeight_Tank2_dt'] = df['Height Tank 2'].diff() / df['time_diff_seconds']

# Shift the derivative columns up by one row
df['dHeight_Tank1_dt'] = df['dHeight_Tank1_dt'].shift(-1)
df['dHeight_Tank2_dt'] = df['dHeight_Tank2_dt'].shift(-1)

# Filter out rows where the time difference is greater than 10 seconds
df = df[df['time_diff_seconds'] <= 10]

# Filter out rows where the CV signal is 0
df = df[df['PWM Value'] != 0]

# Filter out rows where the Tank Heights are misread
df = df[df['Height Tank 2'] > 10]

# Filter out rows where the derivatives are too large due to bad data
df = df[abs(df['dHeight_Tank1_dt']) <= 50]
df = df[abs(df['dHeight_Tank2_dt']) <= 50]

# Drop last data point since no derivative and reset indexes
df = df.reset_index(drop=True)
df = df.drop(df.index[-1])

With the data processed, we can train the GPs.

In [3]:
# Function to create and train a Gaussian Process model
def train_gp(X, y):
    kernel = Matern(nu = 0.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1, random_state=42)
    gp.fit(X.values, y.values)
    return gp

X = df[['PWM Value', 'Height Tank 1', 'Height Tank 2']]
y1 = df['dHeight_Tank1_dt']
y2 = df['dHeight_Tank2_dt']

gp_model_dh1 = train_gp(X, y1)
gp_model_dh2 = train_gp(X, y2)

# Save the trained GP models to files
joblib.dump(gp_model_dh1, 'GP_Models/gp_model_dh1.pkl')
joblib.dump(gp_model_dh2, 'GP_Models/gp_model_dh2.pkl')

# Load the saved GP models from files (verification of appropriate save)
gp_model_dh1_loaded = joblib.load('GP_Models/gp_model_dh1.pkl')
gp_model_dh2_loaded = joblib.load('GP_Models/gp_model_dh2.pkl')

In [4]:
# Make predictions for dh1
df['h1_der_predictions'] = gp_model_dh1_loaded.predict(X)

# Make predictions for dh2
df['h2_der_predictions'] = gp_model_dh2_loaded.predict(X)

c:\Users\micha\anaconda3\envs\online_update_controls\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(
c:\Users\micha\anaconda3\envs\online_update_controls\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


In [5]:
df

,index,Timestamp,PWM Value,Height Tank 1,Height Tank 2,time_diff_seconds,dHeight_Tank1_dt,dHeight_Tank2_dt,h1_der_predictions,h2_der_predictions
0,1,2024-10-08 12:35:33,10.000000,219,210,2.0,0.000000,-1.000000,-6.437950e-11,-1.000000
1,2,2024-10-08 12:35:35,15.000000,219,208,2.0,-0.333333,-2.000000,-3.333333e-01,-2.000000
2,3,2024-10-08 12:35:38,20.000000,218,202,3.0,0.500000,-2.000000,5.000000e-01,-2.000000
3,4,2024-10-08 12:35:40,25.000000,219,198,2.0,-0.500000,-2.000000,-5.000000e-01,-2.000000
4,5,2024-10-08 12:35:42,30.000000,218,194,2.0,0.000000,-0.666667,1.377187e-11,-0.666667
...,...,...,...,...,...,...,...,...,...,...
649,662,2024-10-17 13:09:57,1.000000,222,349,3.0,3.000000,-2.000000,3.000000e+00,-2.000000
650,663,2024-10-17 13:10:00,25.484667,231,343,3.0,8.333333,-0.333333,8.333333e+00,-0.333333
651,664,2024-10-17 13:10:03,100.376000,256,342,3.0,-3.750000,-0.250000,-3.750000e+00,-0.250000
652,665,2024-10-17 13:10:07,17.891333,241,341,4.0,0.666667,-0.333333,6.666667e-01,-0.333333


In [6]:
MSE = np.sum((df['dHeight_Tank2_dt'] - df['h2_der_predictions'])**2)/len(df)
MSE

2.58273047870904e-19

In [7]:
# df['h2_der_diff'] = (df['dHeight_Tank2_dt'] - df['h2_der_predictions'])
# df = df[(df['h2_der_diff'] >= -.01) & (df['h2_der_diff'] <= 0.01)].reset_index(drop=True)
